In [118]:
import smbclient
import json
import cv2
from pathlib import Path
import numpy as np
import zipfile

def write_to_YOLO(dataset_folder, img_url, val=False, test=False):
    img_name = Path(img_url).name

    with smbclient.open_file(img_url, 'rb') as f:
        arr = np.asarray(bytearray(f.read()), dtype=np.uint8)
        image = cv2.imdecode(arr, -1)
        (h, w) = image.shape[:2]
        image = cv2.resize(image, (256, 256), interpolation=cv2.INTER_AREA)

    if val:
        with open('labels_05_13_20_thermal/labels_05_13_20_thermal/' + img_name.replace('.jpg', '.txt'), 'r') as fd:
            data = fd.read()
            x,y,width,height = [float(i) for i in data.strip('\n').strip(' ').split(' ') if i.isdigit()]

            x_center = (x + (width / 2 )) / w
            y_center = (y + (height / 2)) / h
            width = width / w
            height = height / h
            klass = 0
            with open(dataset_folder + '/labels/val/' + img_name.replace('.jpg', '.txt'), 'w+') as f:
                f.write(' '.join([str(i) for i in [klass, x_center, y_center, width, height]]))
        # save the images
        cv2.imwrite(dataset_folder + '/images/val/' + img_name, image)

    else:
        with open('labels_05_13_20_thermal/labels_05_13_20_thermal/' + img_name.replace('.jpg', '.txt'), 'r') as fd:
            data = fd.read()
            x,y,width,height = [float(i) for i in data.strip('\n').strip(' ').split(' ') if i.isdigit()]

            x_center = (x + (width / 2 )) / w
            y_center = (y + (height / 2)) / h
            width = width / w
            height = height / h
            klass = 0
            with open(dataset_folder + '/labels/train/' + img_name.replace('.jpg', '.txt'), 'w+') as f:
                f.write(' '.join([str(i) for i in [klass, x_center, y_center, width, height]]))
        # save the images
        cv2.imwrite(dataset_folder + '/images/train/' + img_name, image)
        

def mount_smb(username, password, dataset_folder, train_imgs, val_imgs):
    # Optional - specify the default credentials to use on the global config object
    smbclient.ClientConfig(username=username, password=password)

    # Optional - register the credentials with a server (overrides ClientConfig for that server)
    smbclient.register_session("192.168.152.34", username=username, password=password)
            
    train_count = 0
    val_count = 0
    for root, dirs, files in smbclient.walk(r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/', 'r'):
        break
    for di in dirs:
        for _, _, files in smbclient.walk(r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/' + di + '/thermal_images/frames/'):
            break
        for file in files:
            if file.replace('.jpg', '.txt') in train_imgs:
                train_count += 1
                write_to_YOLO(dataset_folder, r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/' + di + '/thermal_images/frames/' + file)
            elif file.replace('.jpg', '.txt') in val_imgs:
                val_count += 1
                write_to_YOLO(dataset_folder, r'\\192.168.152.34/DroneData/MatrixSpaceLabeled/05_13_20/Yuneec_Typhoon/2020-05-07-20-27-06/' + di + '/thermal_images/frames/' + file, val=True)
                
    print(train_count)
    print(val_count)


#     print('saving training images...')
#     for img in json_data[1000:2600]:
#         write_to_YOLO(img)

#     print('saving validation images...')
#     for img in json_data[2600:3050]:
#         write_to_YOLO(img, val=True)

#     print('saving testing images...')
#     for img in json_data[3050:3450]:
#         write_to_YOLO(img, test=True)


In [119]:
# import zipfile
# with zipfile.ZipFile('labels_thermal.zip', 'r') as zip_ref:
#     zip_ref.extractall('labels_thermal')
files = glob.glob('labels_05_13_20_thermal/labels_05_13_20_thermal/*.txt')
print(len(files))
username = input('username:')
password = input('password:')
mount_smb(username, password, 'yolov3/labels_05_13_20_thermal', [Path(f).name for f in files[:6700]], [Path(f).name for f in files[6700:]])


7441
6700
741


In [85]:
import glob

def unpack_label(file, trainorval):
    with open(file, 'r+') as fd:
        data = fd.read()
        x,y,width,height = [float(i) for i in data.strip('\n').strip(' ').split(' ') if i.isdigit()]
        x_center = (x + width / 2 )
        y_center = (y + height / 2)
        width = width
        height = height
        klass = 0

    with open('yolov3/thermal_images/labels/' + trainorval + Path(file).name, 'w+') as f:
        f.write(' '.join([str(i) for i in [x_center, y_center, width, height]]))
    
    
files = glob.glob('labels_thermal/labels_thermal/*.txt')
print(len(files))
for file in files[:6700]:
    unpack_label(file, 'train/')
for file in files[6700:]:
    unpack_label(file, 'val/')

7441


In [64]:
Path(files[0]).name

'image_5606.txt'

In [111]:
import os
def clear_dir(directory):
    files = glob.glob(directory)
    for f in files:
        os.remove(f)

clear_dir('yolov3/thermal_images/images/train/*.jpg')
clear_dir('yolov3/thermal_images/images/val/*.jpg')
clear_dir('yolov3/thermal_images/labels/train/*.txt')
clear_dir('yolov3/thermal_images/labels/val/*.txt')

In [101]:
clear_dir('labels_thermal/labels_thermal/*.txt')

In [114]:
import zipfile
with zipfile.ZipFile('labels_05_13_20_thermal.zip', 'r') as zip_ref:
    zip_ref.extractall('labels_05_13_20_thermal')